In [8]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

In [9]:
rho = 998          # Density of water in kg/m^3
sigma = 0.0728     # Surface tension in N/m
mu = 0.001         # Dynamic viscosity in Pa.s
P_inf = 101325     # Ambient pressure (air) in Pascals
P_v = 2330         # Vapor pressure of water in Pascals
R0 = 1e-6          # Initial radius of the bubble (1 micron)
gamma = 1.4        # Polytropic index (air-like gas inside bubble)
P_b0 = P_inf       # Initial bubble pressure (equal to ambient)

**Initial Conditions**

𝑅(0)
=	R0 = 1e-6 m (1 µm) ->	Initial bubble radius

Wall velocity: 𝑅˙(0) = 0 m/s	Bubble starts at rest

t_span = (0, 1e-4)  -> Simulate for 100 microseconds

In [10]:
def P_b(R):
    if R <= 0:
        return P_b0
    return P_b0 * (R0 / R)**(3 * gamma)

Assumes adiabatic compression/expansion of the gas inside the bubble

In [11]:
#Rayleigh–Plesset ODE system: [R, R_dot] --> [R_dot, R_ddot]
def rayleigh_plesset(t, y):
    R, R_dot = y
    if R <= 0:
        return [0, 0]  # Stop everything if radius is invalid

    Pb = P_b(R)
    term1 = (Pb - P_inf - 2 * sigma / R - 4 * mu * R_dot / R) / rho
    R_ddot = (term1 - (3/2) * R_dot**2) / R
    return [R_dot, R_ddot]

In [12]:
#Initial conditions
y0 = [R0, 0]  # Initial radius and initial rate of change (R_dot)

#Time range to simulate (0 to 100 microseconds)
t_span = (0, 1e-2)
t_eval = np.linspace(*t_span, 1000)

solution = solve_ivp(rayleigh_plesset, t_span, y0, t_eval=t_eval, method='RK45')

solve_ivp uses an adaptive Runge–Kutta method (RK45) to numerically integrate the ODE

In [13]:
# Final time and radius
final_time = solution.t[-1]              # Last time point
final_radius = solution.y[0][-1]         # Last radius value
final_wall_velocity = solution.y[1][-1]  # Last dR/dt

# Print in readable units
print(f"Final time: {final_time:.2e} s")
print(f"Final radius: {final_radius*1e6:.2f} µm")
print(f"Final wall velocity: {final_wall_velocity:.2f} m/s")


Final time: 1.00e-02 s
Final radius: 0.78 µm
Final wall velocity: 0.00 m/s


In [14]:
#Plotting the result
# plt.figure(figsize=(8, 5))
# plt.plot(solution.t * 1e4, solution.y[0] * 1e6, color='purple')
# plt.title('Bubble Radius vs Time')
# plt.xlabel('Time (μs)')
# plt.ylabel('Bubble Radius (μm)')
# plt.grid(True)
# plt.show()